### ConversationTokenBuffeMemory
- 대화의 히스토리를 메모리에 보관
- 대화의 개수가 아닌 토큰의 길이를 사용해 대화내용을 내보내줌

In [19]:
from dotenv import load_dotenv

load_dotenv()

True

In [20]:
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationTokenBufferMemory

llm = ChatOpenAI(
    temperature=0,
    model='gpt-4o'
)

memory = ConversationTokenBufferMemory(
    # 토큰수 계산을 위해 , 실제 사용되는 LLM 과 토큰화 방식을 일치시키기 위해
    llm=llm,
    max_token_limit=500,
    return_messages=True
)

In [21]:
memory.save_context(
    inputs={
        "human": "안녕하세요, 피자 한 판 주문하고 싶은데요. 어떤 피자가 있나요?"
    },
    outputs={
        "ai": "안녕하세요! 저희 매장의 인기 메뉴로는 페퍼로니, 치즈, 하와이안, 콤비네이션 피자가 있습니다. 어떤 피자를 선호하시나요?"
    },
)
memory.save_context(
    inputs={"human": "페퍼로니 피자로 할게요. 크기는 어떻게 되나요?"},
    outputs={
        "ai": "페퍼로니 피자는 M(25cm), L(32cm), XL(40cm) 사이즈로 제공됩니다. 크기를 선택해 주세요."
    },
)
memory.save_context(
    inputs={"human": "L 사이즈로 주문할게요. 도우는 어떤 종류가 있나요?"},
    outputs={
        "ai": "도우는 일반, 씬, 치즈크러스트 중에서 선택하실 수 있습니다. 어떤 도우로 하시겠어요?"
    },
)
memory.save_context(
    inputs={"human": "치즈크러스트로 할게요. 토핑 추가는 가능한가요?"},
    outputs={
        "ai": "네, 추가 토핑으로 모짜렐라 치즈, 올리브, 버섯, 피망, 양파를 선택하실 수 있습니다. 추가하실 토핑이 있으신가요?"
    },
)
memory.save_context(
    inputs={"human": "모짜렐라 치즈 추가로 주문할게요. 얼마인가요?"},
    outputs={
        "ai": "L사이즈 페퍼로니 피자(치즈크러스트) + 모짜렐라 치즈 추가는 총 28,000원입니다. 주문을 진행할까요?"
    },
)
memory.save_context(
    inputs={"human": "네, 주문할게요. 배달 시간은 얼마나 걸리나요?"},
    outputs={
        "ai": "주문이 완료되었습니다. 현재 예상 배달 시간은 약 40분입니다. 주문 현황은 문자메시지로 전송해 드리겠습니다. 감사합니다!"
    },
)

In [22]:
print(memory.load_memory_variables({})['history'])

[HumanMessage(content='안녕하세요, 피자 한 판 주문하고 싶은데요. 어떤 피자가 있나요?', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요! 저희 매장의 인기 메뉴로는 페퍼로니, 치즈, 하와이안, 콤비네이션 피자가 있습니다. 어떤 피자를 선호하시나요?', additional_kwargs={}, response_metadata={}), HumanMessage(content='페퍼로니 피자로 할게요. 크기는 어떻게 되나요?', additional_kwargs={}, response_metadata={}), AIMessage(content='페퍼로니 피자는 M(25cm), L(32cm), XL(40cm) 사이즈로 제공됩니다. 크기를 선택해 주세요.', additional_kwargs={}, response_metadata={}), HumanMessage(content='L 사이즈로 주문할게요. 도우는 어떤 종류가 있나요?', additional_kwargs={}, response_metadata={}), AIMessage(content='도우는 일반, 씬, 치즈크러스트 중에서 선택하실 수 있습니다. 어떤 도우로 하시겠어요?', additional_kwargs={}, response_metadata={}), HumanMessage(content='치즈크러스트로 할게요. 토핑 추가는 가능한가요?', additional_kwargs={}, response_metadata={}), AIMessage(content='네, 추가 토핑으로 모짜렐라 치즈, 올리브, 버섯, 피망, 양파를 선택하실 수 있습니다. 추가하실 토핑이 있으신가요?', additional_kwargs={}, response_metadata={}), HumanMessage(content='모짜렐라 치즈 추가로 주문할게요. 얼마인가요?', additional_kwargs={}, response_metadata={})

In [23]:
from langchain.chains import ConversationChain

conversation = ConversationChain(
    llm=llm,
    memory=memory
)

response = conversation.predict(
    input="처음부터 다시 너가 추천해줬던 모든 피자에 대해 설명해줘"
)

print(response)

안녕하세요! 제가 추천했던 피자들에 대해 자세히 설명해드릴게요.

1. **페퍼로니 피자**: 클래식한 맛을 자랑하는 페퍼로니 피자는 얇게 썬 페퍼로니 소시지가 듬뿍 올라가 있어 짭짤하고 매콤한 맛이 특징입니다. 치즈와 토마토 소스가 조화를 이루어 많은 분들이 좋아하는 메뉴입니다.

2. **치즈 피자**: 치즈 피자는 다양한 종류의 치즈가 듬뿍 올라가 있어 고소하고 부드러운 맛을 자랑합니다. 특히 치즈 애호가들에게 인기가 많으며, 기본적인 맛을 선호하시는 분들께 추천드립니다.

3. **하와이안 피자**: 하와이안 피자는 달콤한 파인애플과 짭짤한 햄이 어우러져 독특한 맛을 냅니다. 달콤함과 짭짤함의 조화가 매력적이며, 색다른 피자를 시도해보고 싶으신 분들께 좋습니다.

4. **콤비네이션 피자**: 콤비네이션 피자는 다양한 토핑이 올라가 있어 풍부한 맛을 제공합니다. 보통 페퍼로니, 소시지, 양파, 피망, 버섯 등이 포함되어 있어 여러 가지 맛을 한 번에 즐길 수 있습니다.

각 피자는 M(25cm), L(32cm), XL(40cm) 사이즈로 제공되며, 도우는 일반, 씬, 치즈크러스트 중에서 선택하실 수 있습니다. 추가 토핑도 가능하니, 취향에 맞게 선택해보세요!
